In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import data_process
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import cv2

In [2]:
data_process.data_to_tfrecord(imagepath='../data/image_mobilenet/', outputpath='../data/face_data_multi.tfrecord', mode='multi')

이미지를 tfrecord에 저장중: 100%|██████████| 452/452 [00:01<00:00, 388.26it/s]

이미지 갯수:  452
[4, 0, 0, 3, 3, 3, 4, 5, 3, 4, 6, 0, 2, 4, 1, 2, 4, 0, 0, 1, 5, 6, 0, 5, 3, 6, 1, 6, 2, 6, 3, 1, 4, 0, 2, 4, 6, 6, 4, 3, 4, 2, 2, 0, 1, 2, 3, 6, 0, 2, 0, 3, 1, 0, 1, 0, 5, 0, 0, 6, 5, 5, 5, 0, 0, 1, 2, 2, 3, 5, 4, 3, 1, 5, 2, 3, 4, 2, 6, 2, 4, 5, 5, 1, 4, 3, 0, 2, 2, 4, 4, 5, 5, 4, 5, 3, 2, 1, 3, 2, 0, 1, 0, 4, 4, 6, 3, 0, 4, 0, 0, 0, 2, 2, 2, 5, 3, 5, 0, 1, 0, 1, 0, 0, 6, 2, 1, 1, 3, 3, 0, 3, 4, 6, 2, 4, 0, 4, 4, 0, 6, 6, 3, 3, 5, 0, 0, 4, 5, 3, 3, 2, 3, 3, 2, 3, 3, 1, 5, 6, 3, 2, 3, 0, 0, 6, 6, 3, 1, 3, 2, 1, 4, 0, 0, 6, 0, 3, 2, 6, 1, 6, 3, 0, 6, 5, 2, 0, 1, 0, 5, 2, 1, 3, 5, 2, 4, 3, 1, 0, 2, 3, 2, 5, 6, 1, 6, 2, 0, 3, 2, 0, 3, 1, 0, 6, 3, 2, 4, 6, 2, 2, 0, 2, 2, 1, 0, 4, 6, 0, 5, 3, 6, 6, 0, 1, 5, 4, 4, 3, 3, 2, 2, 3, 3, 3, 0, 4, 4, 0, 3, 0, 3, 3, 0, 2, 2, 6, 5, 5, 0, 1, 1, 5, 3, 2, 5, 2, 6, 4, 4, 6, 2, 5, 1, 0, 3, 2, 3, 6, 3, 4, 6, 1, 0, 6, 0, 1, 0, 4, 1, 1, 6, 6, 5, 5, 3, 1, 6, 4, 6, 5, 3, 1, 0, 3, 2, 0, 4, 0, 0, 5, 5, 5, 1, 1, 2, 5, 1, 1, 2, 0, 5, 3, 4, 3, 6, 0, 3,

In [ ]:
# _filenames = '../data/face_data_7.tfrecord'
# raw_dataset = tf.data.TFRecordDataset(filenames)
# raw_dataset
# # Create a dictionary describing the features.
# image_feature_description = {
#     'image': tf.io.FixedLenFeature([], tf.string),    
#     'class': tf.io.FixedLenFeature([], tf.int64),
#     'landmark_points': tf.io.FixedLenFeature([], tf.float32),    
# }

# def _parse_image_function(example_proto): 
#   # Parse the input tf.train.Example proto using the dictionary above.  
#   return tf.io.parse_single_example(example_proto, image_feature_description)

# parsed_image_dataset = raw_dataset.map(_parse_image_function)
# parsed_image_dataset

In [ ]:
# for image_features in parsed_image_dataset:
#     print(image_features)
#     image_raw = image_features['class'].numpy()
# print(image_raw)
# # display.display(display.Image(data=image_raw))

In [8]:
@tf.function
def landmark_tfrecord_reader(example):
    feature_description = {"image": tf.io.VarLenFeature(dtype=tf.string),
                           "class": tf.io.VarLenFeature(dtype=tf.int64),
                           "landmark_points": tf.io.VarLenFeature(dtype=tf.float32)}

    inputs = {}
    example = tf.io.parse_single_example(example, feature_description)
    image_raw = tf.sparse.to_dense(example["image"])[0]
    inputs['image'] = tf.io.decode_png(image_raw, channels=3)
    image_class = tf.sparse.to_dense(example["class"])
    inputs['image_landmarks'] = tf.cast(tf.sparse.to_dense(example["landmark_points"]), tf.float32)
    return inputs, image_class

In [9]:
def image_scaling(x, y):
    x['image'] = tf.cast(x['image'], tf.float32) / 255.
    # x 480pixel, y 640pixel
    x['image_landmarks'] = tf.cast(x['image_landmarks'], tf.float32) / 640.
    return x, y

In [10]:
filenames = '../data/face_data_multi.tfrecord'
ds = tf.data.TFRecordDataset(filenames).map(landmark_tfrecord_reader)
ds = ds.map(image_scaling).batch(32)
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [11]:
lds = list(ds)

In [12]:
lds[0][1]

<tf.Tensor: shape=(32, 7), dtype=int64, numpy=
array([[0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 